In [1]:
import os, shutil, json, logging, re
import pandas as pd
from pathlib import Path
from typing import List
from datetime import datetime
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import FileResponse
from openpyxl import load_workbook
from openai import OpenAI
from dotenv import load_dotenv
import nest_asyncio, uvicorn

# ───────────────────────────────────────────
# 환경 설정
# ───────────────────────────────────────────
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TEMPLATE_PATH = Path(os.getenv("TEMPLATE_PATH", r"C:\Users\che98\Downloads\수익률표(Brief Version)_1.0_템플릿.xlsx"))
client = OpenAI(api_key=OPENAI_API_KEY)

# ───────────────────────────────────────────
# FastAPI 앱 및 로깅 설정
# ───────────────────────────────────────────
app = FastAPI()
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")
log = logging.getLogger(__name__)
nest_asyncio.apply()

# ───────────────────────────────────────────
# 유틸 함수
# ───────────────────────────────────────────
def find_last_row(ws):
    for row in reversed(range(1, ws.max_row + 1)):
        if any(cell.value for cell in ws[row]):
            return row
    return 2

# ───────────────────────────────────────────
# GPT 호출 - 첫 배치
# ───────────────────────────────────────────
def call_gpt_first_batch(batch_text: str, batch_idx: int) -> tuple[List[dict], List[str]]:
    prompt = f"""
다음은 부실채권 요약 텍스트 목록입니다. 

**1단계: 헤더 정보 추출**
먼저 이 데이터에서 컬럼 헤더나 필드명을 찾아서 리스트로 추출해주세요.

**2단계: 데이터 정리**
각 항목을 아래 항목들에 맞게 JSON으로 정리해주세요:

- 숫자는 쉼표(,) 없이 숫자형으로, 소수점은 유지해주세요
- **이자율은 %를 제거하고 소수점 형태로 출력해주세요** (예: 11.323%이면 0.11323으로)
- 같은 필드가 한 줄에 여러 번 등장하면 합산
- 항목이 중복되면 적절히 구분해서 필드에 배정
- 특히, 가지급금은 "여신성 가지급금"으로, 미수이자는 "이자금액" 또는 "채권 권리 합계"와 구분
- 채권최고액은 보통 가장 큰 금액이며 그 항목 전용으로

필드: 주소, 호수, 검토일, 차주, 반장님 전달, 매각여부, 경매/공매, 담당기관, 물건종류, 세대수, 대지면적, 전용면적(합계), 대출잔액, 이자금액, 연체이자율, 대출원금, 채권 권리 합계, 채권최고액, 감정가, 선순위 합계, 여신성 가지급금

입력:
{batch_text}

응답 형식:
```json
{{
  "headers": ["감지된 헤더1", "감지된 헤더2", ...],
  "data": [
    {{"주소": "값", "호수": "값", ...}},
    {{"주소": "값", "호수": "값", ...}}
  ]
}}
```
"""
    with open(f"prompt_batch_{batch_idx+1}_first.txt", "w", encoding="utf-8") as f:
        f.write(prompt)

    try:
        res = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            max_tokens=4096
        )
        msg = res.choices[0].message.content.strip()
        match = re.search(r"```json\s*(.*?)```", msg, re.DOTALL)
        json_str = match.group(1).strip() if match else msg
        result = json.loads(json_str)
        return result.get("data", []), result.get("headers", [])
    except Exception as e:
        with open(f"gpt_error_batch_{batch_idx+1}_first.txt", "w", encoding="utf-8") as f:
            f.write(msg if 'msg' in locals() else str(e))
        raise RuntimeError(f"GPT 응답 파싱 실패: {e}")

# ───────────────────────────────────────────
# GPT 호출 - 이후 배치
# ───────────────────────────────────────────
def call_gpt_with_headers(batch_text: str, batch_idx: int, headers: List[str]) -> List[dict]:
    headers_str = ", ".join(headers) if headers else "헤더 정보 없음"
    prompt = f"""
다음은 부실채권 요약 텍스트 목록입니다. 

**참고 헤더 정보:**
이전 배치에서 감지된 헤더: {headers_str}

이 헤더 정보를 참고하여 각 항목을 아래 항목들에 맞게 JSON으로 정리해주세요:

- 숫자는 쉼표(,) 없이 숫자형으로, 소수점은 유지해주세요
- **이자율은 %를 제거하고 소수점 형태로 출력해주세요** (예: 11.323%이면 0.11323으로)
- 같은 필드가 한 줄에 여러 번 등장하면 합산
- 항목이 중복되면 적절히 구분해서 필드에 배정
- 특히, 가지급금은 "여신성 가지급금"으로, 미수이자는 "이자금액" 또는 "채권 권리 합계"와 구분
- 채권최고액은 보통 가장 큰 금액이며 그 항목 전용으로
- 출력은 반드시 ```json 으로 감싸진 JSON 리스트로

필드: 주소, 호수, 검토일, 차주, 반장님 전달, 매각여부, 경매/공매, 담당기관, 물건종류, 세대수, 대지면적, 전용면적(합계), 대출잔액, 이자금액, 연체이자율, 대출원금, 채권 권리 합계, 채권최고액, 감정가, 선순위 합계, 여신성 가지급금

입력:
{batch_text}

응답 형식은 반드시 ```json 블록으로 시작해서 JSON 리스트만 포함해주세요.
"""

    with open(f"prompt_batch_{batch_idx+1}_with_headers.txt", "w", encoding="utf-8") as f:
        f.write(prompt)

    try:
        res = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            max_tokens=4096
        )
        msg = res.choices[0].message.content.strip()
        match = re.search(r"```json\s*(.*?)```", msg, re.DOTALL)
        json_str = match.group(1).strip() if match else msg
        return json.loads(json_str)
    except Exception as e:
        with open(f"gpt_error_batch_{batch_idx+1}_with_headers.txt", "w", encoding="utf-8") as f:
            f.write(msg if 'msg' in locals() else str(e))
        raise RuntimeError(f"GPT 응답 파싱 실패: {e}")

# ───────────────────────────────────────────
# Excel 데이터 추출 및 분할
# ───────────────────────────────────────────
def extract_batches_from_excel(file_path: str, batch_size: int = 10) -> List[List[str]]:
    xls = pd.ExcelFile(file_path)
    all_rows = []
    for sheet_name in xls.sheet_names:
        try:
            df = pd.read_excel(file_path, sheet_name=sheet_name, dtype=str, header=1).fillna("")
            for _, row in df.iterrows():
                text = f"[{sheet_name}] " + " | ".join([f"{col}:{row[col]}" for col in df.columns])
                all_rows.append(text)
        except Exception as e:
            log.warning(f"⚠️ 시트 '{sheet_name}' 처리 실패: {e}")
    return [all_rows[i:i+batch_size] for i in range(0, len(all_rows), batch_size)]

# ───────────────────────────────────────────
# Excel 결과 삽입 (수정됨)
# ───────────────────────────────────────────
def insert_batch_to_excel(batch_data: List[dict], output_path: str):
    wb = load_workbook(output_path)
    ws = wb["List"]
    col_order = [cell.value for cell in ws[2] if cell.value]
    start_col = 2
    start_row = find_last_row(ws) + 1
    
    # 금액 필드 (이자율 제외)
    money_fields = [
        "감정가", "대출잔액", "이자금액", "대출원금", "채권 권리 합계",
        "채권최고액", "선순위 합계", "여신성 가지급금"
    ]
    
    # 이자율 필드 (별도 처리)
    interest_rate_fields = ["연체이자율"]
    
    # 숫자 필드 (면적, 세대수 등)
    numeric_fields = ["세대수", "대지면적", "전용면적(합계)"]

    for item in batch_data:
        for col_idx, col_name in enumerate(col_order):
            raw = item.get(col_name, "")
            
            if col_name in money_fields:
                # 금액 필드: 숫자만 추출하여 float 변환
                try:
                    value = float(re.sub(r"[^\d.]", "", str(raw))) if re.search(r"\d", str(raw)) else None
                except:
                    value = None
                    
            elif col_name in interest_rate_fields:
                # 이자율 필드: 숫자만 추출 후 100으로 나누기
                try:
                    # 모든 문자 제거하고 숫자만 추출
                    cleaned_num = re.sub(r"[^\d.]", "", str(raw))
                    if cleaned_num and re.search(r"\d", cleaned_num):
                        # 100으로 나누어서 정상 이자율로 변환
                        value = float(cleaned_num) / 1
                        log.info(f"🔍 이자율 처리: '{raw}' → '{cleaned_num}' → '{value}'")
                    else:
                        value = None
                except:
                    value = None
                    
            elif col_name in numeric_fields:
                # 일반 숫자 필드: 쉼표 제거하고 숫자 변환
                try:
                    cleaned = re.sub(r"[^\d.]", "", str(raw))
                    value = float(cleaned) if cleaned and re.search(r"\d", cleaned) else None
                except:
                    value = None
                    
            else:
                # 텍스트 필드: 그대로 유지
                value = raw
                
            ws.cell(row=start_row, column=start_col + col_idx, value=value)
        start_row += 1

    wb.save(output_path)

# ───────────────────────────────────────────
# 전체 프로세스 실행
# ───────────────────────────────────────────
def process_file(upload_path: str, output_path: str):
    batches = extract_batches_from_excel(upload_path)
    detected_headers = []
    
    for idx, batch in enumerate(batches):
        try:
            batch_text = "\n\n".join(batch)
            if idx == 0:
                data, headers = call_gpt_first_batch(batch_text, idx)
                detected_headers = headers
                with open("detected_headers.json", "w", encoding="utf-8") as f:
                    json.dump(detected_headers, f, ensure_ascii=False, indent=2)
            else:
                data = call_gpt_with_headers(batch_text, idx, detected_headers)
            insert_batch_to_excel(data, output_path)
            log.info(f"✅ 배치 {idx+1}: {len(data)}행 추가 완료")
        except Exception as e:
            log.error(f"❌ 배치 {idx+1} 실패: {e}")
            continue

# ───────────────────────────────────────────
# 업로드 엔드포인트
# ───────────────────────────────────────────
@app.post("/upload/")
async def upload_and_process(file: UploadFile = File(...)):
    upload_dir = Path("uploaded")
    upload_dir.mkdir(exist_ok=True)
    file_path = upload_dir / file.filename
    with open(file_path, "wb") as f:
        shutil.copyfileobj(file.file, f)
    log.info(f"업로드됨: {file_path}")

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    out = Path(f"작성본_{Path(file.filename).stem}_{timestamp}.xlsx")
    shutil.copyfile(TEMPLATE_PATH, out)

    try:
        process_file(str(file_path), str(out))
    except Exception as e:
        log.error("처리 실패", exc_info=e)
        raise HTTPException(status_code=500, detail=f"처리 실패: {e}")

    return FileResponse(out)

# ───────────────────────────────────────────
# 로컬 실행
# ───────────────────────────────────────────
if __name__ == "__main__":
    uvicorn.run(app, host="127.0.0.1", port=8000)

INFO:     Started server process [3236]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:62925 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:62925 - "GET /.well-known/appspecific/com.chrome.devtools.json HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:62925 - "GET /openapi.json HTTP/1.1" 200 OK


2025-07-24 16:11:54,624 [INFO] 업로드됨: uploaded\2025 3차 채권매각명세.xlsx
2025-07-24 16:12:31,548 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-24 16:13:10,408 [INFO] 🔍 이자율 처리: '0.11323' → '0.11323' → '0.11323'
2025-07-24 16:13:10,411 [INFO] 🔍 이자율 처리: '0.0957' → '0.0957' → '0.0957'
2025-07-24 16:13:10,412 [INFO] 🔍 이자율 처리: '0.0831' → '0.0831' → '0.0831'
2025-07-24 16:13:10,413 [INFO] 🔍 이자율 처리: '0.0937' → '0.0937' → '0.0937'
2025-07-24 16:13:10,415 [INFO] 🔍 이자율 처리: '0.0903' → '0.0903' → '0.0903'
2025-07-24 16:13:10,416 [INFO] 🔍 이자율 처리: '0.0808' → '0.0808' → '0.0808'
2025-07-24 16:13:10,417 [INFO] 🔍 이자율 처리: '0.0878' → '0.0878' → '0.0878'
2025-07-24 16:13:10,418 [INFO] 🔍 이자율 처리: '0.09114' → '0.09114' → '0.09114'
2025-07-24 16:13:41,719 [INFO] ✅ 배치 1: 8행 추가 완료
2025-07-24 16:14:14,527 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-24 16:15:00,940 [INFO] 🔍 이자율 처리: '0.08499' → '0.08499' → '0.0849